In [ ]:
import random
import json

In [ ]:
!python3 -m pip install AWSIoTPythonSDK

In [ ]:
AWS_REGION = "us-east-1"
WORK_FOLDER = "."
PRIVATE_KEY_OUTFILE = "iot_client.private.key"
PUBLIC_KEY_OUTFILE = "iot_client.public.key"
CERT_PEM_OUTFILE = "iot_client.cert.pem"
GREENGRASSS_CORENAME = "toradex2"
THIS_IOT_CLIENT_NAME = "mqttClient_{}".format(random.randint(0,100))
AWS_REGION = "us-east-1"

In [ ]:
!wget -q https://www.amazontrust.com/repository/AmazonRootCA1.pem

In [ ]:
import boto3

client = boto3.client('iot')

response = client.create_keys_and_certificate(
    setAsActive=True
)

In [ ]:
client.create_thing(thingName=THIS_IOT_CLIENT_NAME)
client.attach_thing_principal(
            thingName=THIS_IOT_CLIENT_NAME, principal=response['certificateArn'])

policy_name="policy-{}".format(THIS_IOT_CLIENT_NAME)

policy=json.dumps(
    {
        "Version": "2012-10-17",
        "Statement": [{
            "Effect": "Allow",
            "Action": [
                # "iot:*"
                "iot:Connect",
                "iot:Publish",
                "iot:Receive",
                "iot:Subscribe"
            ],
            "Resource": [
                "arn:aws:iot:{0}:*:*".format(AWS_REGION)
            ]
        }]
    }
)

client.create_policy(policyName=policy_name, policyDocument=policy)

client.attach_principal_policy(policyName=policy_name, principal=response['certificateArn'])

In [ ]:
with open('private.key', 'w') as file:
    file.write(response["keyPair"]["PrivateKey"])

with open('cert.pem', 'w') as file:
    file.write(response["certificatePem"])

In [ ]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient
import boto3
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

client = boto3.client('iot', region_name=AWS_REGION)
endpoint = client.describe_endpoint(
    endpointType='iot:Data-ATS'
)

# Custom MQTT message callback
def subscriptionCallback(client, userdata, message):
    payload = message.payload
    nparr = np.frombuffer(payload, np.uint8) #.reshape(h, w, nb_planes )
    image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    plt.imshow(image)
    
awsIoTMqttClient = AWSIoTMQTTClient("myClientID")
awsIoTMqttClient.configureEndpoint(endpoint.get('endpointAddress'), 8883)
awsIoTMqttClient.configureCredentials( "AmazonRootCA1.pem", "private.key", "cert.pem")
awsIoTMqttClient.connect()
awsIoTMqttClient.subscribe("mli/predictions/{}".format(CORENAME), 1, subscriptionCallback)
key_pressed = input('Press ENTER to stop subscription')
awsIoTMqttClient.disconnect()